In [ ]:
# import keras
# from keras.datasets import cifar10
# from keras.models import Model, Sequential
# from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
# from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
# from keras.layers import Concatenate
# from keras.optimizers import Adam
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from keras import regularizers

from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.layers import Input, Flatten, Dense,GlobalMaxPooling2D


: 

In [ ]:
import tensorflow as tf

In [ ]:
# Hyperparameters
batch_size = 32
num_classes = 10
epochs = 10
l = 40
num_filter = 18
compression = 0.5


In [ ]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]

# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170500096/170498071 [==============================] - 11s 0us/step


In [5]:
#https://machinelearningmastery.com/how-to-develop-a-cnn-from-scratch-for-cifar-10-photo-classification/
# scale pixels
def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm
X_train, X_test=prep_pixels(X_train,X_test)

In [6]:
# Dense Block
def denseblock(input, num_filter = 12):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_4 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)

        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_4])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter = 12):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False,kernel_regularizer = regularizers.l2() ,padding='same')(relu)

    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Pooling = layers.AveragePooling2D(pool_size=(2,2))(relu)

    fc_conv=Conv2D(num_classes,kernel_size=[2,2])(Pooling)
    output = Activation('softmax')(fc_conv)
    flat=Flatten()(output)

    return flat

In [7]:
num_filter = 36
l = 14
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(36, (3,3), use_bias=False ,padding='same')(input)

First_Block = denseblock(First_Conv2D, num_filter)
First_Transition = transition(First_Block, num_filter)

Second_Block = denseblock(First_Transition, num_filter)
Second_Transition = transition(Second_Block, num_filter)

Third_Block = denseblock(Second_Transition, num_filter)
Third_Transition = transition(Third_Block, num_filter)

Last_Block = denseblock(Third_Transition,  num_filter)
output=output_layer(Last_Block)
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 36)   972         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 36)   144         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 36)   0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [8]:
#determine Loss function and Optimizer
sgd = SGD(lr=0.01, decay=1e-5, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
img = ImageDataGenerator(width_shift_range=0.5,rotation_range=15, height_shift_range=0.5,shear_range=0.2,horizontal_flip=True,zoom_range=0.2)
img.fit(X_train)
train_gen = img.flow(X_train, y_train,batch_size=32,shuffle=True)
history = model.fit(train_gen,steps_per_epoch=X_train.shape[0]/32,epochs=299, validation_data=(X_test, y_test), verbose=1)

Epoch 1/299
1563/1562 [==============================] - 302s 193ms/step - loss: 2.2041 - accuracy: 0.2609 - val_loss: 1.7722 - val_accuracy: 0.3613
Epoch 2/299
1563/1562 [==============================] - 298s 191ms/step - loss: 1.8063 - accuracy: 0.3511 - val_loss: 1.6149 - val_accuracy: 0.4308
Epoch 3/299
1563/1562 [==============================] - 299s 191ms/step - loss: 1.6635 - accuracy: 0.4093 - val_loss: 2.0754 - val_accuracy: 0.3637
Epoch 4/299
1563/1562 [==============================] - 299s 191ms/step - loss: 1.5584 - accuracy: 0.4547 - val_loss: 1.9116 - val_accuracy: 0.4185
Epoch 5/299
1563/1562 [==============================] - 298s 191ms/step - loss: 1.4551 - accuracy: 0.5039 - val_loss: 2.0680 - val_accuracy: 0.4039
Epoch 6/299
1563/1562 [==============================] - 298s 191ms/step - loss: 1.3814 - accuracy: 0.5339 - val_loss: 1.8815 - val_accuracy: 0.4587
Epoch 7/299
1563/1562 [==============================] - 298s 191ms/step - loss: 1.3215 - accuracy: 0.5564

In [ ]:
history.save_weights(os.path.join(path, 'first_30_epoch.h5'))